# Data Processing

In [12]:
from IPython.display import display as display_nb
import pandas as pd

In [1]:
catalog

In [2]:
catalog.list()

['companies', 'reviews', 'shuttles', 'my_dataset', 'parameters']

## `preprocess_companies`

In [41]:
companies = catalog.load('companies')
companies.head()

2021-11-11 17:44:06,619 - kedro.io.data_catalog - INFO - Loading data from `companies` (CSVDataSet)...


,id,company_rating,company_location,total_fleet_count,iata_approved
0,35029,100%,Niue,4.0,f
1,30292,67%,Anguilla,6.0,f
2,19032,67%,Russian Federation,4.0,f
3,8238,91%,Barbados,15.0,t
4,30342,NaN,Sao Tome and Principe,2.0,t


In [123]:
companies.duplicated().sum()

26998

In [4]:
companies.shape

(77096, 5)

In [7]:
companies.dtypes

id                     int64
company_rating        object
company_location      object
total_fleet_count    float64
iata_approved         object
dtype: object

In [5]:
companies.isna().mean()

id                   0.000000
company_rating       0.387945
company_location     0.248132
total_fleet_count    0.000091
iata_approved        0.000091
dtype: float64

In [18]:
with pd.option_context('display.max_rows', None):
    display_nb(companies[['company_location']].drop_duplicates())

,company_location
0,Niue
1,Anguilla
2,Russian Federation
3,Barbados
4,Sao Tome and Principe
5,Faroe Islands
6,Micronesia
7,Rwanda
8,Uzbekistan
10,Kiribati


In [21]:
companies[['iata_approved']].drop_duplicates()

,iata_approved
0,f
3,t
8655,NaN


### Data checks

- `id`
     - Check if all integers
- `company_rating`
    - Check if follow the pattern '[0-9]{1,2}%'
- `company_location`
    - Check if actual location (?)
- `total_fleet_count`
    - Check if all float
- `iata_approved`
    - Check if values are only `f` or `t`


### Data Processing

- `company_rating`
    - convert to float
- `iata_approved`
    - convert to boolean

In [27]:
companies_proc = companies.copy()

In [26]:
companies_proc['company_rating'].str.replace('%', '').astype(float)

0        100.0
1         67.0
2         67.0
3         91.0
4          NaN
         ...  
77091    100.0
77092      NaN
77093      NaN
77094     80.0
77095     98.0
Name: company_rating, Length: 77096, dtype: float64

In [149]:
class CompanyPreprocessor:
    
    def __init__(self, copy=True):
        self.copy = copy
    
    @staticmethod
    def _parse_percentage(x: pd.Series)-> pd.Series:
        return x.str.replace('%', '').astype(float)/100
    
    @staticmethod
    def _is_true(x: pd.Series)-> pd.Series:
        return pd.Series(x.map({'t': True, 'f': False}), dtype='boolean')
    
    @staticmethod
    def _deduplicate(df: pd.DataFrame) -> pd.DataFrame:
        return df.drop_duplicates().reset_index(drop=True)
    
    def __call__(self, companies: pd.DataFrame)-> pd.DataFrame:
        if self.copy:
            companies = companies.copy()
            
        companies = self._deduplicate(companies)
        companies['company_rating'] = self._parse_percentage(companies['company_rating'])
        companies['iata_approved'] = self._is_true(companies['iata_approved'])
        
        return companies
        

In [150]:
preprocess_companies = CompanyPreprocessor()
preprocess_companies(companies)

,id,company_rating,company_location,total_fleet_count,iata_approved
0,35029,1.00,Niue,4.0,False
1,30292,0.67,Anguilla,6.0,False
2,19032,0.67,Russian Federation,4.0,False
3,8238,0.91,Barbados,15.0,True
4,30342,NaN,Sao Tome and Principe,2.0,True
...,...,...,...,...,...
50093,1742,NaN,Greenland,1.0,False
50094,15249,NaN,Marshall Islands,1.0,False
50095,44431,NaN,NaN,1.0,False
50096,25724,NaN,NaN,1.0,False


## `preprocess_shuttles`

In [151]:
shuttles = catalog.load('shuttles')

2021-11-11 19:05:33,354 - kedro.io.data_catalog - INFO - Loading data from `shuttles` (ExcelDataSet)...


In [152]:
shuttles.head()

,id,shuttle_location,shuttle_type,engine_type,engine_vendor,engines,passenger_capacity,cancellation_policy,crew,d_check_complete,moon_clearance_complete,price,company_id
0,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,f,f,"$1,325.0",35029
1,36260,Anguilla,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,t,f,"$1,780.0",30292
2,57015,Russian Federation,Type V5,Quantum,ThetaBase Services,1.0,2,moderate,0.0,f,f,"$1,715.0",19032
3,14035,Barbados,Type V5,Plasma,ThetaBase Services,3.0,6,strict,3.0,f,f,"$4,770.0",8238
4,10036,Sao Tome and Principe,Type V2,Plasma,ThetaBase Services,2.0,4,strict,2.0,f,f,"$2,820.0",30342


In [130]:
shuttles.duplicated().sum()

0

In [128]:
shuttles.isna().mean()

id                         0.000000
shuttle_location           0.000000
shuttle_type               0.000000
engine_type                0.000000
engine_vendor              0.000000
engines                    0.000506
passenger_capacity         0.000000
cancellation_policy        0.000000
crew                       0.001933
d_check_complete           0.000000
moon_clearance_complete    0.000000
price                      0.000000
company_id                 0.000000
dtype: float64

In [67]:
shuttles.dtypes

id                           int64
shuttle_location            object
shuttle_type                object
engine_type                 object
engine_vendor               object
engines                    float64
passenger_capacity           int64
cancellation_policy         object
crew                       float64
d_check_complete            object
moon_clearance_complete     object
price                       object
company_id                   int64
dtype: object

In [90]:
def show_unique_vals(df):
    '''Show nice formatted unique values in each column'''
    
    uniq_df = (df.apply(lambda col: col.unique())
               .reset_index()
               .rename(columns={'index': 'col', 0:'uniq_vals'})
              )
        
    uniq_df['num_uniq'] = uniq_df['uniq_vals'].apply(len)
    
    with pd.option_context('display.max_colwidth', None):
        display_nb(uniq_df)

In [82]:
from spaceflights.utils import show_unique_vals # Nice!

In [91]:
cols = ['shuttle_type', 'engine_type', 'engine_vendor', 'cancellation_policy',
        'd_check_complete', 'moon_clearance_complete']
show_unique_vals(shuttles[cols])

,col,uniq_vals,num_uniq
0,shuttle_type,"[Type V5, Type V2, Type F5, Type O3, Type V7, Type G0, Type S7, Type Z6, Type F1, Type E3, Type Y4, Type X3, Type W5, Type L7, Type I5, Type Z1, Type T7, Type B7, Type K2, Type N0, Type D1, Type D4, Type P6, Type A7, Type Y7, Type U1, Type L0, Type O7, Type O0, Type S1, Type W4, Type F6, Type D2, Type I2, Type I1, Type O1, Type L2, Type R1, Type C2, Type I7, Type A4, Type U0]",42
1,engine_type,"[Quantum, Plasma, Nuclear]",3
2,engine_vendor,"[ThetaBase Services, Warwick Technology Multinational, Banks, Wood and Phillips, MCW Global, SIT Technology Unlimited]",5
3,cancellation_policy,"[strict, moderate, flexible]",3
4,d_check_complete,"[f, t]",2
5,moon_clearance_complete,[f],1


In [92]:
shuttles.head()

,id,shuttle_location,shuttle_type,engine_type,engine_vendor,engines,passenger_capacity,cancellation_policy,crew,d_check_complete,moon_clearance_complete,price,company_id
0,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,f,f,"$1,325.0",35029
1,36260,Anguilla,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,t,f,"$1,780.0",30292
2,57015,Russian Federation,Type V5,Quantum,ThetaBase Services,1.0,2,moderate,0.0,f,f,"$1,715.0",19032
3,14035,Barbados,Type V5,Plasma,ThetaBase Services,3.0,6,strict,3.0,f,f,"$4,770.0",8238
4,10036,Sao Tome and Principe,Type V2,Plasma,ThetaBase Services,2.0,4,strict,2.0,f,f,"$2,820.0",30342


### Data Processing

- `price`
    - convert to float
- `d_check_complete`
    - convert to bool
- `moon_clearance_complete`
    - convert to bool

In [147]:
class ShuttlesPreprocessor:
    
    def __init__(self, copy=True):
        self.copy = copy
    
    @staticmethod
    def _is_true(x: pd.Series)-> pd.Series:
        return pd.Series(x.map({'t': True, 'f': False}), dtype='boolean')
    
    @staticmethod
    def _parse_money(x: pd.Series)-> pd.Series:
        return x.str.replace(r'[^0-9]', '', regex=True)
    
    @staticmethod
    def _deduplicate(df: pd.DataFrame)-> pd.DataFrame:
        return df.drop_duplicates().reset_index(drop=True)
        
    
    def __call__(self, shuttles: pd.DataFrame)-> pd.DataFrame:
        if self.copy:
            shuttles = shuttles.copy()
        
        shuttles = self._deduplicate(shuttles)
        shuttles['price'] = self._parse_money(shuttles['price'])
        shuttles['d_check_complete'] = self._is_true(shuttles['d_check_complete'])
        shuttles['moon_clearance_complete'] = self._is_true(shuttles['moon_clearance_complete'])
        
        return shuttles

In [148]:
preprocess_shuttles = ShuttlesPreprocessor()
preprocess_shuttles(shuttles)

,id,shuttle_location,shuttle_type,engine_type,engine_vendor,engines,passenger_capacity,cancellation_policy,crew,d_check_complete,moon_clearance_complete,price,company_id
0,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,False,13250,35029
1,36260,Anguilla,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,True,False,17800,30292
2,57015,Russian Federation,Type V5,Quantum,ThetaBase Services,1.0,2,moderate,0.0,False,False,17150,19032
3,14035,Barbados,Type V5,Plasma,ThetaBase Services,3.0,6,strict,3.0,False,False,47700,8238
4,10036,Sao Tome and Principe,Type V2,Plasma,ThetaBase Services,2.0,4,strict,2.0,False,False,28200,30342
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77091,4368,Barbados,Type V5,Quantum,ThetaBase Services,2.0,4,flexible,2.0,True,False,41070,6654
77092,2983,Bouvet Island (Bouvetoya),Type F5,Quantum,ThetaBase Services,1.0,1,flexible,1.0,True,False,11690,8000
77093,69684,Micronesia,Type V5,Plasma,ThetaBase Services,0.0,2,flexible,1.0,True,False,19100,14296
77094,21738,Uzbekistan,Type V5,Plasma,ThetaBase Services,1.0,2,flexible,1.0,True,False,21700,27363


## Create Master Table

In [135]:
catalog.list()

['companies', 'reviews', 'shuttles', 'my_dataset', 'parameters']

In [136]:
reviews = catalog.load('reviews')
reviews.head(1)

2021-11-11 18:28:04,887 - kedro.io.data_catalog - INFO - Loading data from `reviews` (CSVDataSet)...


,shuttle_id,review_scores_rating,review_scores_comfort,review_scores_amenities,review_scores_trip,review_scores_crew,review_scores_location,review_scores_price,number_of_reviews,reviews_per_month
0,63561,97.0,10.0,9.0,10.0,10.0,9.0,10.0,133,1.65


In [137]:
reviews.duplicated().sum()

0

In [138]:
shuttles.head(1)

,id,shuttle_location,shuttle_type,engine_type,engine_vendor,engines,passenger_capacity,cancellation_policy,crew,d_check_complete,moon_clearance_complete,price,company_id
0,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,f,f,"$1,325.0",35029


In [139]:
companies.head(1)

,id,company_rating,company_location,total_fleet_count,iata_approved
0,35029,100%,Niue,4.0,f


In [140]:
def create_master_table(
    shuttles: pd.DataFrame,
    companies: pd.DataFrame,
    reviews: pd.DataFrame)-> pd.DataFrame:
    '''Combines all data to create a master table'''
    
    rated_shuttles = shuttles.merge(reviews, left_on='id', right_on='shuttle_id')
    master_table = rated_shuttles.merge(companies, left_on='company_id', right_on='id').dropna()

    return master_table

In [142]:
master_table = create_master_table(shuttles, companies.drop_duplicates(), reviews)

In [144]:
master_table

,id_x,shuttle_location,shuttle_type,engine_type,engine_vendor,engines,passenger_capacity,cancellation_policy,crew,d_check_complete,...,review_scores_crew,review_scores_location,review_scores_price,number_of_reviews,reviews_per_month,id_y,company_rating,company_location,total_fleet_count,iata_approved
0,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,f,...,10.0,9.0,10.0,133,1.65,35029,100%,Niue,4.0,f
1,53260,Niue,Type V5,Quantum,"Banks, Wood and Phillips",1.0,2,strict,1.0,f,...,10.0,9.0,10.0,37,0.48,35029,100%,Niue,4.0,f
2,51019,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,flexible,1.0,f,...,10.0,9.0,9.0,10,0.15,35029,100%,Niue,4.0,f
3,53898,Niue,Type V5,Plasma,ThetaBase Services,3.0,5,strict,3.0,f,...,10.0,9.0,10.0,11,0.21,35029,100%,Niue,4.0,f
4,36260,Anguilla,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,t,...,9.0,9.0,9.0,3,0.09,30292,67%,Anguilla,6.0,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76454,39094,Russian Federation,Type F5,Quantum,ThetaBase Services,1.0,2,strict,1.0,f,...,10.0,10.0,10.0,1,1.00,42904,70%,Russian Federation,2.0,t
76497,20330,Uzbekistan,Type V5,Quantum,ThetaBase Services,1.0,2,flexible,1.0,f,...,10.0,10.0,10.0,1,1.00,5701,100%,Costa Rica,1.0,t
76526,16445,Nicaragua,Type V5,Plasma,ThetaBase Services,1.0,1,flexible,3.0,f,...,10.0,10.0,10.0,3,3.00,13728,100%,Pakistan,1.0,f
76584,76469,Bouvet Island (Bouvetoya),Type V5,Quantum,ThetaBase Services,1.0,2,moderate,1.0,f,...,10.0,10.0,10.0,1,1.00,41714,100%,Lebanon,1.0,f
